In [1]:
import argparse
import numpy as np
import os
import random

# ray related utils
import ray
from ray import tune
from ray.rllib.agents import ppo, dqn
from ray.rllib.models import ModelCatalog
from ray.tune.logger import pretty_print

# trinity related utils
import solinv.tyrell.spec as S
import solinv.tyrell.dsl as D
from solinv.tyrell.interpreter import InvariantInterpreter
from solinv.environment import InvariantEnvironment
# from solinv.model import InvariantGRU, SimpleInvariantGRU
from solinv.model import InvariantTGN

if __name__ == "__main__":
    spec = S.parse_file("./dsls/abstract0.tyrell")
    start_type = spec.get_type("Expr")
    interpreter = InvariantInterpreter()
    env_config = {
        "spec": spec,
        "start_type": start_type,
        "max_step": 6,
        "contract_path": "../SolidTypes/test/regression/good/mint_MI.sol",
        # options are: 0.4.26, 0.5.17, 0.6.12
        "solc_version": "0.5.17",
        "interpreter": interpreter
    }
    # need to construct the vocab first to provide parameters for nn
    tmp_environment = InvariantEnvironment(config=env_config)

    ray.init(local_mode=True)
    # ModelCatalog.register_custom_model("invariant_gru", InvariantGRU)
    # ModelCatalog.register_custom_model("invariant_gru", SimpleInvariantGRU)
    ModelCatalog.register_custom_model("invariant_tgn", InvariantTGN)

    ppo_config = ppo.DEFAULT_CONFIG.copy()
    rl_config = {
        "env": InvariantEnvironment,
        "env_config": env_config,
        "model": {
            "custom_model": "invariant_tgn",
            "custom_model_config": {
                "contract": {
                    "igraph": tmp_environment.contract_igraph_ids,
                },
                "nn": {
                    "num_token_embeddings": len(tmp_environment.token_list),
                    "token_embedding_dim": 16,

                    "contract_out_dim": 32,

                    "invariant_hidden_dim": 64,
                    "invariant_out_dim": 32,

                    "action_out_dim": len(tmp_environment.action_list),
                }
            },
        },
        "num_workers": 1,
        "framework": "torch",
    }
    ppo_config.update(rl_config)
    agent = ppo.PPOTrainer(env=InvariantEnvironment, config=ppo_config)

2021-09-28 16:20:33,823	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265
2021-09-28 16:20:35,703	INFO ppo.py:158 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2021-09-28 16:20:35,707	INFO trainer.py:726 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2021-09-28 16:20:35,869	WARNING worker.py:498 -- `ray.get_gpu_ids()` will always return the empty list when called from the driver. This is because Ray does not manage GPU allocations to the driver process.
2021-09-28 16:20:36,426	WARNING catalog.py:546 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2021-09-28 16:20:36,525	WARNING worker.py:498 -- `ray.get_gpu_ids()` will always return the empty list when called from the driver. This is because Ray does not mana

# input_dict is: SampleBatch(['obs', 'new_obs', 'actions', 'prev_actions', 'rewards', 'prev_rewards', 'dones', 'infos', 'eps_id', 'unroll_id', 'agent_index', 't', 'obs_flat'])
# input_dict[infos] is: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
# This is a dummy batch.
# input_dict is: SampleBatch(['obs', 'seq_lens', 'obs_flat'])
# This is a dummy batch.
# input_dict is: SampleBatch(['obs', 'new_obs', 'actions', 'prev_actions', 'rewards', 'prev_rewards', 'dones', 'infos', 'eps_id', 'unroll_id', 'agent_index', 't', 'vf_preds', 'action_dist_inputs', 'action_prob', 'action_logp', 'advantages', 'value_targets', 'obs_flat'])
# input_dict[infos] is: tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
# This is a dummy batch.
# input_dict is: SampleBatch(['obs', 'new_obs', 'actions', 'prev_actions', 're

In [56]:
tmp_environment.contract_igraph_ids.es["token"]

[33,
 27,
 34,
 30,
 2,
 28,
 31,
 13,
 23,
 25,
 30,
 30,
 2,
 28,
 2,
 31,
 28,
 2,
 31,
 13,
 23,
 25,
 30,
 2,
 29,
 30,
 2,
 28,
 2,
 31,
 32,
 30,
 2,
 24,
 2,
 26,
 29,
 30,
 2,
 24,
 2,
 26,
 28,
 2,
 31,
 32,
 13,
 14,
 15,
 16,
 13]

In [3]:
dir(agent)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_allow_unknown_configs',
 '_allow_unknown_subkeys',
 '_before_evaluate',
 '_close_logfiles',
 '_create_logger',
 '_default_config',
 '_env_id',
 '_episodes_total',
 '_evaluate',
 '_experiment_id',
 '_export_model',
 '_implements_method',
 '_init',
 '_iteration',
 '_iterations_since_restore',
 '_local_ip',
 '_logdir',
 '_make_workers',
 '_monitor',
 '_name',
 '_open_logfiles',
 '_override_all_subkeys_if_type_changes',
 '_policy_class',
 '_register_if_needed',
 '_restored',
 '_result_logger',
 '_stderr_context',
 '_stderr_file',
 '_stderr_fp',
 '_stderr_logging_handler',
 '_stderr_stream',

In [14]:
type(agent)

ray.rllib.agents.trainer_template.PPO

In [5]:
agent.get_policy().model

InvariantTGN(
  (token_embedding): Embedding(43, 16)
  (contract_conv): TransformerConv(16, 32, heads=2)
  (invariant_projection): Linear(in_features=16, out_features=64, bias=True)
  (invariant_encoder): GRU(64, 32, batch_first=True)
  (action_branch): Linear(in_features=32, out_features=11, bias=True)
  (value_branch): Linear(in_features=32, out_features=1, bias=True)
)

In [7]:
agent.get_policy()

In [12]:
agent.get_policy().model.__hash__()

8789327323639

In [8]:
dir(agent.get_policy().model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_contract_features',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_invariant_features',
 '_is_full_backward_hook',
 '_last_output',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forwar

In [15]:
import torch

In [17]:
a = torch.tensor([1])

In [18]:
a

tensor([1])

In [20]:
type(a)

torch.Tensor

In [19]:
dir(a)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rfloordiv__',
 '__rmul__',
 '__rpow__',
 '__rshi

In [47]:
import torch
class Tensor(torch.Tensor):
    @staticmethod
    def __new__(cls, *args, info=None, **kwargs):
        return super().__new__(cls, *args, **kwargs)
    
    def __init__(self, *args, info=None, **kwargs):
        super().__init__() # optional
        self.info = info

In [48]:
Tensor([1], info="eee").info

'eee'

In [27]:
dir(torch.Tensor)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__cuda_array_interface__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rfloordiv__',
 '__r

In [51]:
torch.tensor(["a"])

ValueError: too many dimensions 'str'

In [65]:
all((torch.tensor([1]) == 0).tolist())

False